In [1]:
import numpy as np
import pandas as pd
from scripts.drosophila import depth_first_search

In [2]:
conn_m=np.load('/media/maria/DATA1/Documents/NeuralData/conn_dir_drosophila.npy')[:1000,:1000]

In [3]:
n_nodes=1000
visited_arr=np.zeros((n_nodes,))
visited_arr=depth_first_search(conn_m,0,visited_arr)


In [5]:
print(visited_arr)

[1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [7]:
conn_connected=conn_m[visited_arr!=0,visited_arr!=0]
print(conn_connected.shape)

(919,)


In [9]:
from brian2 import *
N = conn_connected.shape[0]
tau = 10*ms
vr = -70*mV
vt = -40*mV
eqs = '''
dv/dt = -v/tau : volt
'''
w = 1*mV
G = NeuronGroup(N, eqs, threshold='v>vt', reset='v=vr')
S = Synapses(G, G, model='w : volt', on_pre='v += w')
for i in range(0, conn_connected.shape[0]):
    for j in range(0,conn_connected.shape[1]):
        if conn_m[i,j]!=0:
            S.connect(i=i, j=j)

IndexError: tuple index out of range